In [19]:
import os
import pandas as pd

In [20]:
""" 
The code outputs Pandas DataFrames of the images of the tier1 training set

DataFame descriptions:

data_images = {
                id: (String) id of relevent image (same for both pre and post disaster image)
                image_id: (String) unique id for each image
                location: (String) disaster title
                phase: ("pre" or "post")
                disaster: (String) disaster done to the building
                image_path: (String) path name to grab image later
              }

data_buildings = {
                polygon_id: (String) unique id for each polygon annotation 
                image_id: (String) unique id for the relevent image
                phase: ("pre" or "post")
                damage: (String) damage type done to the building (no damage, minor damage, major damage, destroyed)
                polygon: (Polygon Object) unique polygon outlining builiding in the image
                 }
              
"""

IMAGE_DATA_PATH = "./geotiffs/tier1/images/"
POLYGON_DATA_PATH = "./geotiffs/tier1/labels/"

image_entries = []
polygon_entries = []

for json_file in os.listdir(POLYGON_DATA_PATH):
    if json_file.endswith(".json"):
        with open(os.path.join(POLYGON_DATA_PATH, json_file), 'r') as f:
            data = json.load(f)
        
        id = None
        image_id = None
        location = None
        phase = None
        disaster = None
        image_path = None
        
        image_name = data['metadata']['img_name']
        image_path = os.path.join(IMAGE_DATA_PATH, image_name.replace(".png", ".tif"))

        base_name = image_name.replace(".png", "")
        parts = base_name.split("_")
        
        id = data['metadata']['id']
        
        image_id = parts[1]
        phase = parts[2]

        location = data['metadata']['disaster']
        disaster = data['metadata']['disaster_type']

        image_entries.append({
            "id": id,
            "image_id": image_id,
            "location": location,
            "phase": phase,
            "disaster": disaster,
            "image_path": image_path
        })

        for id in data['features']['xy']:

            polygon_id = None
            damage = None
            polygon = None

            polygon_id = id["properties"]["uid"]
            
            if phase == "pre":
                damage = "pre"
            elif phase == "post":
                damage = id["properties"]["subtype"]

            polygon = id["wkt"]
            
            polygon_entries.append({
                "polygon_id": polygon_id,
                "image_id": image_id,
                "phase": phase,
                "damage": damage,
                "polygon": polygon
            })

data_images = pd.DataFrame(image_entries)
data_buildings = pd.DataFrame(polygon_entries)